<a href="https://colab.research.google.com/github/yasirfaizahmed/ML_Notebooks/blob/master/NumberPlate_detection_using_YOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# license plate object detection using YOLOv8

#### dataset : https://huggingface.co/datasets/keremberke/license-plate-object-detection


In [2]:
! pip install datasets ultralytics

  Using cached datasets-3.5.0-py3-none-any.whl.metadata (19 kB)
  Using cached ultralytics-8.3.107-py3-none-any.whl.metadata (37 kB)
  Using cached aiohttp-3.11.16-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
  Using cached matplotlib-3.10.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached torch-2.6.0-cp312-cp312-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached torchvision-0.21.0-cp312-cp312-manylinux1_x86_64.whl.metadata (6.1 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
Using cached datasets-3.5.0-py3-none-any.whl (491 kB)
Using cached ultralytics-8.3.107-py3-none-any.whl (974 kB)
Using cached aiohttp-3.11.16-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.7 MB)
Using cached huggingface_hub-0.30.2-py3-none-any.whl (481 kB)
Usi

In [3]:
from datasets import load_dataset

ds = load_dataset("keremberke/license-plate-object-detection", "full")

/home/xd/Documents/python_codes/ML_Notebooks/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 882/882 [00:00<00:00, 43973.19 examples/s]


In [4]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import cv2
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/xd/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
df_train = ds['train'].to_pandas()
df_val = ds['validation'].to_pandas()
df_test = ds['test'].to_pandas()
df_train.info()
df_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6176 entries, 0 to 6175
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   image_id  6176 non-null   int64 
 1   image     6176 non-null   object
 2   width     6176 non-null   int32 
 3   height    6176 non-null   int32 
 4   objects   6176 non-null   object
dtypes: int32(2), int64(1), object(2)
memory usage: 193.1+ KB


,image_id,image,width,height,objects
0,6152,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,472,303,"{'id': [6388], 'area': [2604], 'bbox': [[261.0..."
1,5500,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,472,303,"{'id': [5719], 'area': [742], 'bbox': [[263.0,..."
2,1188,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,400,240,"{'id': [1231], 'area': [2701], 'bbox': [[147.0..."
3,3524,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,540,405,"{'id': [3651], 'area': [800], 'bbox': [[217.0,..."
4,5049,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,472,303,"{'id': [5242], 'area': [2112], 'bbox': [[145.0..."


In [6]:
# Load a pre-trained YOLOv8 model
model = YOLO('yolov8n.pt')  # 'yolov8n.pt' is a small model, you can choose others like 'yolov8

100%|██████████| 6.25M/6.25M [00:00<00:00, 61.1MB/s]


In [7]:
import os
import cv2
print(df_train["objects"][0])


{'id': array([6388]), 'area': array([2604]), 'bbox': array([array([        261,         203,          93,          28], dtype=float32)], dtype=object), 'category': array([0])}


In [8]:
# Output directories
image_dir = 'dataset/images/train'
label_dir = 'dataset/labels/train'
os.makedirs(image_dir, exist_ok=True)
os.makedirs(label_dir, exist_ok=True)

for idx, row in df_train.iterrows():
    # Convert byte string to image
    image = np.frombuffer(row['image'].get('bytes'), dtype=np.uint8)
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)

    if image is not None:
      image_height, image_width = image.shape[:2]
    else:
      continue

    # Save image
    image_path = os.path.join(image_dir, f'image_{idx}.jpg')
    cv2.imwrite(image_path, image)

    # Convert bbox to YOLO format
    # Extract bounding box and class ID
    bbox = row['objects'].get("bbox")  # [x_min, y_min, width, height]
    class_id = 0 # Class ID

    # Normalize bbox coordinates
    x_min, y_min, width, height = bbox[0]
    x_center = (x_min + width / 2) / image_width
    y_center = (y_min + height / 2) / image_height
    bbox_width = width / image_width
    bbox_height = height / image_height

    # Save label file
    label_path = os.path.join(label_dir, f'image_{idx}.txt')
    with open(label_path, 'w') as f:
        f.write(f'{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n')

In [9]:
# Output directories
image_dir = 'dataset/images/val'
label_dir = 'dataset/labels/val'
os.makedirs(image_dir, exist_ok=True)
os.makedirs(label_dir, exist_ok=True)

for idx, row in df_val.iterrows():
    # Convert byte string to image
    image = np.frombuffer(row['image'].get('bytes'), dtype=np.uint8)
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)

    if image is not None:
      image_height, image_width = image.shape[:2]
    else:
      continue

    # Save image
    image_path = os.path.join(image_dir, f'image_{idx}.jpg')
    cv2.imwrite(image_path, image)

    # Convert bbox to YOLO format
    # Extract bounding box and class ID
    bbox = row['objects'].get("bbox")  # [x_min, y_min, width, height]
    class_id = 0  # Class ID


    # Normalize bbox coordinates
    x_min, y_min, width, height = bbox[0]
    x_center = (x_min + width / 2) / image_width
    y_center = (y_min + height / 2) / image_height
    bbox_width = width / image_width
    bbox_height = height / image_height

    # Save label file
    label_path = os.path.join(label_dir, f'image_{idx}.txt')
    with open(label_path, 'w') as f:
        f.write(f'{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n')

In [13]:
# Train the model using the YAML file
results = model.train(data="dataset.yaml", epochs=75, imgsz=640, batch=16)

Ultralytics 8.3.107 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060, 11915MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset.yaml, epochs=75, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width

100%|██████████| 755k/755k [00:00<00:00, 17.4MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 51.7MB/s]


...

      75/75      3.74G     0.9038     0.3253     0.9845         16        640: 100%|██████████| 386/386 [00:39<00:00,  9.66it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 56/56 [00:04<00:00, 13.06it/s]

                   all       1765       1765      0.985      0.978      0.992      0.741



75 epochs completed in 0.931 hours.
Optimizer stripped from /home/xd/Documents/python_codes/ML_Notebooks/runs/detect/train4/weights/last.pt, 6.2MB
Optimizer stripped from /home/xd/Documents/python_codes/ML_Notebooks/runs/detect/train4/weights/best.pt, 6.2MB

Validating /home/xd/Documents/python_codes/ML_Notebooks/runs/detect/train4/weights/best.pt...
Ultralytics 8.3.107 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060, 11915MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 56/56 [00:04<00:00, 12.22it/s]


                   all       1765       1765      0.985      0.978      0.993      0.742
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to /home/xd/Documents/python_codes/ML_Notebooks/runs/detect/train4
